In [1]:
import numpy as np
import pandas as pd

from datetime import date
from datetime import datetime

from sklearn.neighbors import KernelDensity

import seaborn as sns
import random as r

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
# For downsampling
from sklearn.utils import resample,shuffle
r_seed1, r_seed2, r_seed3, r_seed4, r_seed5, n_est= 533, 2608, 1581, 2218, 641, 63
np.random.seed(r_seed1)

In [2]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

d:\python\anaconda\envs\datamininglab\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)


###test safekeeping dataframe
df_main_test=df[df.set=="test"]

df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


In [12]:
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

In [13]:
df_train

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp
0,1,2455,597,48.78,2013-08-01,1,in-kind items and services,food and beverage,nan,nan,nan,no,no third party payment,nan,nan,nan,covered,nan,nan,nan,device or medical supply,nan,nan,pacemakers,nan,nan,nan,nan,nan,train,thippeswamy,h,murthy,nan,fayetteville,ga,30214-2114,united states,nan,allopathic & osteopathic physicians|internal m...,ga,nan,nan,nan,nan,"medtronic vascular, inc.",mn,united states
1,2,1534,593,122.43,2013-08-01,1,in-kind items and services,food and beverage,nan,nan,nan,no,no third party payment,no,nan,nan,covered,1529370001,nan,nan,drug or biological,nan,nan,vascepa,nan,nan,nan,nan,nan,train,romeo,nan,jones,nan,brooklyn,ny,11217-1889,united states,nan,allopathic & osteopathic physicians|internal m...,ny,nan,nan,nan,nan,amarin pharma inc.,nj,united states
3,4,550,760,13.68,2013-08-01,1,in-kind items and services,education,nan,nan,nan,no,no third party payment,nan,nan,nan,covered,nan,nan,nan,drug or biological,nan,nan,acthar,nan,nan,nan,nan,nan,train,robyn,g,young,nan,alameda,ca,94501-6447,united states,nan,allopathic & osteopathic physicians|psychiatry...,ca,nan,nan,nan,nan,questcor pharmaceuticals,ca,united states
4,5,1710,826,17.17,2013-08-01,1,in-kind items and services,food and beverage,nan,nan,nan,no,no third party payment,nan,nan,nan,covered,5254420431,nan,nan,drug or biological,nan,nan,generess,nan,nan,nan,nan,nan,train,lanetta,nan,anderson,nan,memphis,tn,38104-2934,united states,nan,nan,tn,nan,nan,nan,nan,actavis pharma inc,nj,united states
5,6,3803,145,12.88,2013-08-01,1,in-kind items and services,food and beverage,nan,nan,nan,no,no third party payment,nan,nan,nan,covered,5024205306,nan,nan,drug or biological,nan,nan,rituxan,nan,nan,nan,nan,nan,train,robert,e,hunt,nan,huntsville,al,35801-4414,united states,nan,allopathic & osteopathic physicians|internal m...,al,ia,oh,nan,nan,"genentech usa, inc.",ca,united states
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402243,1402244,2319,1268,15.90,2019-12-31,1,cash or cash equivalent,food and beverage,nan,nan,nan,no,no third party payment,nan,nan,nan,yes,67546-111-12,67546-111-14,67546-212-21,drug,drug,drug,alinia,alinia,alinia,gastroenterology,gastroenterology,gastroenterology,train,elisa,m,faybush,nan,mesa,az,85204-7286,united states,nan,allopathic & osteopathic physicians|internal m...,az,nan,nan,nan,nan,"romark laboratories, lc",fl,united states
1402245,1402246,4705,401,11838.32,2019-12-31,2,cash or cash equivalent,royalty or license,nan,nan,nan,no,no third party payment,nan,nan,nan,yes,nan,nan,nan,device,nan,nan,creo fenestrated screw,nan,nan,spine,nan,nan,train,rohit,nan,vasan,nan,tampa,fl,33606-3603,united states,nan,allopathic & osteopathic physicians|neurologic...,fl,nan,nan,nan,nan,"globus medical, inc.",pa,united states
1402246,1402247,1040,1268,9.57,2019-12-31,1,cash or cash equivalent,food and beverage,nan,nan,nan,no,no third party payment,nan,nan,nan,yes,67546-111-12,67546-111-14,67546-212-21,drug,drug,drug,alinia,alinia,alinia,gastroenterology,gastroenterology,gastroenterology,train,lisa,m,rankin,nan,port st lucie,fl,34983,united states,nan,allopathic & 

In [33]:
pd.set_option('display.max_rows', 50)
df_train.sort_values(['Physician_ID','Company_ID'])
company_id_list=df_train.Company_ID.unique()
for comp in company_id_list:
    df_scrap=df_train.query('Company_ID==@comp')
    phys_list=df_scrap.Physician_ID.unique()
    for phys in phys_list:
        if (df_scrap.query('Physician_ID==@phys and Ownership_Indicator=="yes"').shape[0]>0) and (df_scrap.query('Physician_ID==@phys and Ownership_Indicator=="no"').shape[0]>0):
            print(phys)
            print(comp)
    

3827
114
3694
114
4025
114
5991
114
4939
114
2910
114
1161
114
4584
114
278
114
5007
114
3893
114
5966
114
4938
114
5981
114
3669
114
1621
114
1991
114
5739
114
3828
114
2434
114
1856
114
5279
114
5298
114
1735
114
1675
114
403
114
2018
114
3903
114
757
114
4310
114
4935
114
2686
114
5030
114
1082
114
2477
114
5455
114
4014
114
5321
114
913
114
1836
114
5934
114
1422
114
550
114
3461
114
1960
114
3069
114
857
114
2415
114
1471
114
5552
114
2941
114
595
114
4292
114
1746
114
1324
114
3193
114
5869
114
1539
114
93
114
1081
114
152
114
4324
114
3990
114
5540
114
4416
114
4156
114
4143
114
4772
114
1756
114
4131
114
3227
114
3208
114
1582
114
5500
114
5236
114
137
114
1192
114
4678
114
5556
114
2222
114
3897
114
4504
114
3857
114
794
114
682
114
1797
114
5459
114
1093
114
1158
216
3904
37
2897
480
5255
126
3389
126
1266
126
5261
126
5255
878
604
878
1476
878
1158
1171
1232
1222
3904
24
1158
24
4311
65
1034
772
609
772
191
341
4677
686
2106
686
1862
686
734
686
2639
686
3752
686
5150
686
24

KeyboardInterrupt: 

In [36]:
df_train.query('Physician_ID=="3827" and Company_ID == "114"').Ownership_Indicator.value_counts()

no     92
yes    19
Name: Ownership_Indicator, dtype: int64

In [ ]:
###DROPPING RECORDS
df_scrap=df.query('Product_Code_1!="nan" and Product_Name_1=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_2!="nan" and Product_Name_2=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_3!="nan" and Product_Name_3=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)


In [ ]:
###ENCODING CELL

name_list = pd.unique(df[['Product_Name_1', 'Product_Name_2', 'Product_Name_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Name_1'] = df['Product_Name_1'].apply(product_encode)
df['Product_Name_2'] = df['Product_Name_2'].apply(product_encode)
df['Product_Name_3'] = df['Product_Name_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Code_1', 'Product_Code_2', 'Product_Code_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Code_1'] = df['Product_Code_1'].apply(product_encode)
df['Product_Code_2'] = df['Product_Code_2'].apply(product_encode)
df['Product_Code_3'] = df['Product_Code_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Category_1'] = df['Product_Category_1'].apply(product_encode)
df['Product_Category_2'] = df['Product_Category_2'].apply(product_encode)
df['Product_Category_3'] = df['Product_Category_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Type_1', 'Product_Type_2','Product_Type_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Type_1'] = df['Product_Type_1'].apply(product_encode)
df['Product_Type_2'] = df['Product_Type_2'].apply(product_encode)
df['Product_Type_3'] = df['Product_Type_3'].apply(product_encode)


name_list = pd.unique(df[['License_State_1', 'License_State_2', 'License_State_3', 'License_State_4', 'License_State_5']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def license_encode(license_state):
    return name_list_dict.get(license_state)

df['License_State_1']=df['License_State_1'].apply(license_encode)
df['License_State_2']=df['License_State_2'].apply(license_encode)
df['License_State_3']=df['License_State_3'].apply(license_encode)
df['License_State_4']=df['License_State_4'].apply(license_encode)
df['License_State_5']=df['License_State_5'].apply(license_encode)

name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day

columns_encode=['Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes


In [ ]:
###DROPPING COLUMNS
df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province'],inplace=True)

###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Indicator")
df_train.insert((df_train.shape[1]),"Ownership_Indicator",first_col)

first_col = df_test.pop("Ownership_Indicator")
df_test.insert((df_test.shape[1]),"Ownership_Indicator",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)



In [ ]:
##downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=r_seed2)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train,random_state=r_seed3)
test = shuffle(test,random_state=r_seed4)

In [ ]:
###feature and label split
features=train[train.columns[0:(train.shape[1]-3)]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:(test.shape[1]-3)]]
y_test = test['Ownership_Indicator']

###model training
clf = RandomForestClassifier(n_jobs=-1, oob_score = True, n_estimators = n_est, random_state=r_seed5)

clf.fit(features, y)

# Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('RF accuracy: TRAINING', clf.score(features,y))
print ('RF accuracy: TESTING', clf.score(x_test,y_test))

In [ ]:
phys_id_list=df_main_test['Physician_ID']

In [ ]:
predict_set=df_test
predict_set.drop(columns=['Physician_ID','Ownership_Indicator'],inplace=True)

In [ ]:
score = clf.predict(predict_set)

In [ ]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
physUnique=submit_df['Physician_ID'].unique()


In [ ]:

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        

In [ ]:
final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)

In [ ]:
fs_df.to_csv("./submission3.csv",index=False)